## Подготовка данных

In [2]:
import pickle
import numpy as np
with open("../data/ecoli_10_10000_1000.data", "rb") as file:
    data_list = pickle.load(file)
data = data_list[0]
data.train_ans = np.expand_dims(data.train_ans, 1)

In [3]:
print(data.train1.shape, data.train_ans.shape)

(10000, 10, 4) (10000, 1, 4)


In [112]:
import numpy as np
data.train_hmm = np.concatenate((data.train1, data.train_ans), 1)
print(data.train_hmm.shape)
# data.train_hmm = np.concatenate(data.train_hmm, axis = 0)
# print(data.train_hmm.shape)

(10000, 11, 4)


In [113]:
inp = np.zeros((10000, 11), dtype = np.int32)
for i in range(10000):
    for j in range(11):
        inp[i, j] = np.argmax(data.train_hmm[i, j, ...])
        
lengths = np.fromiter(map(len, inp), dtype = np.int32)
        
print(inp.shape)
inp = np.concatenate(inp).reshape(-1, 1)
print(inp.shape)

(10000, 11)
(110000, 1)


In [114]:
test = np.zeros((1000, 10), dtype = np.int32)
for i in range(1000):
    for j in range(10):
        test[i, j] = np.argmax(data.test1[i, j, ...])
        
lengths_test = np.fromiter(map(len, test), dtype = np.int32)
print(test.shape)
test = np.concatenate(test).reshape(-1, 1)
print(test.shape)

(1000, 10)
(10000, 1)


## Самая простая модель - скрытая марковская

https://hmmlearn.readthedocs.io/en/latest/tutorial.html

In [115]:
from hmmlearn.hmm import MultinomialHMM

In [118]:
model = MultinomialHMM(n_iter = 100, n_components=4)

In [119]:
model.fit(inp, lengths)

KeyboardInterrupt: 

In [ ]:
out = model.predict(test, lengths_test)

In [ ]:
max(out)

## Простой пример

In [129]:
X1 = [0, 0, 1, 0, 1]
X2 = [0, 1, 0, 1]
X = np.concatenate([X1, X2]).reshape(-1, 1)
lengths = [len(X1), len(X2)]
model = MultinomialHMM(n_components=2).fit(X, lengths)

In [130]:
model.predict([[0], [0], [1]], [3])

array([1, 1, 0])

https://simple-hohmm.readthedocs.io/en/latest/

In [145]:
inp = np.zeros((10000, 11), dtype = np.int32)
for i in range(10000):
    for j in range(11):
        inp[i, j] = np.argmax(data.train_hmm[i, j, ...])
inp.shape

(10000, 11)

In [133]:
import SimpleHOHMM

In [151]:
possible_states = ['0', '1']
possible_observations = [0, 1, 2 , 3]

In [152]:
from SimpleHOHMM import HiddenMarkovModelBuilder as Builder
builder = Builder()
hmm = builder.build_unsupervised(
        single_states=possible_states,
        all_obs=possible_observations,
        distribution="random",
        highest_order=1
)

In [153]:
# hmm.display_parameters()
hmm.learn(inp, k_smoothing=0.001)
# hmm.display_parameters()

2

In [154]:
hmm.display_parameters()

Starting probabilities (pi):
{'0': 0.5000000000000006, '1': 0.49999999999999944}
Transition probabilities (A):
[0.49993780417781575, 0.500062195822184]
[0.500062195822185, 0.4999378041778153]
Emission probabilities (B):
[0.18163672654690594, 0.18163672654690619, 0.45381963346035203, 0.1816367265469061]
[0.1816367265469064, 0.18163672654690619, 0.4538196334603519, 0.18163672654690627]
